# Sitzung 5 – Multi-Agent-Orchestrator

Demonstriert eine einfache Zwei-Agenten-Pipeline (Forscher -> Redakteur) mit Foundry Local.


### Erklärung: Abhängigkeitsinstallation
Installiert `foundry-local-sdk` und `openai`, die für den Zugriff auf lokale Modelle und Chat-Abschlüsse benötigt werden. Idempotent.


# Szenario
Implementiert ein minimales Orchestrierungsmuster mit zwei Agenten:
- **Forscher-Agent** sammelt prägnante, faktenbasierte Stichpunkte
- **Redakteur-Agent** formuliert diese für eine klare und prägnante Darstellung

Demonstriert gemeinsamen Speicher pro Agent, sequentielle Übergabe von Zwischenergebnissen und eine einfache Pipeline-Funktion. Erweiterbar auf weitere Rollen (z. B. Kritiker, Prüfer) oder parallele Zweige.

**Umgebungsvariablen:**
- `FOUNDRY_LOCAL_ALIAS` - Standardmodell, das verwendet wird (Standard: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Primäres Agentenmodell (überschreibt ALIAS)
- `AGENT_MODEL_EDITOR` - Redakteur-Agentenmodell (Standard: primäres Modell)

**SDK-Referenz:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Funktionsweise:**
1. **FoundryLocalManager** startet automatisch den Foundry Local-Dienst
2. Lädt das angegebene Modell herunter und lädt es (oder verwendet die zwischengespeicherte Version)
3. Stellt einen OpenAI-kompatiblen Endpunkt für die Interaktion bereit
4. Jeder Agent kann ein anderes Modell für spezialisierte Aufgaben verwenden
5. Eingebaute Wiederholungslogik bewältigt vorübergehende Fehler problemlos

**Hauptmerkmale:**
- ✅ Automatische Dienstentdeckung und Initialisierung
- ✅ Modell-Lebenszyklusmanagement (Download, Cache, Laden)
- ✅ OpenAI-SDK-Kompatibilität für vertraute API
- ✅ Unterstützung mehrerer Modelle für Agentenspezialisierung
- ✅ Robuste Fehlerbehandlung mit Wiederholungslogik
- ✅ Lokale Inferenz (keine Cloud-API erforderlich)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Erklärung: Wichtige Importe & Typisierung
Stellt Dataclasses für die Speicherung von Agentennachrichten und Typ-Hinweise zur besseren Verständlichkeit vor. Importiert den Foundry Local Manager und den OpenAI-Client für nachfolgende Aktionen des Agenten.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Erklärung: Modellinitialisierung (SDK-Muster)
Verwendet das Foundry Local Python SDK für eine robuste Modellverwaltung:
- **FoundryLocalManager(alias)** - Startet den Dienst automatisch und lädt das Modell anhand des Alias
- **get_model_info(alias)** - Wandelt den Alias in eine konkrete Modell-ID um
- **manager.endpoint** - Stellt den Dienstendpunkt für den OpenAI-Client bereit
- **manager.api_key** - Stellt den API-Schlüssel bereit (optional für lokale Nutzung)
- Unterstützt separate Modelle für verschiedene Agenten (primär vs. Editor)
- Eingebaute Wiederholungslogik mit exponentiellem Backoff für höhere Ausfallsicherheit
- Verbindungsprüfung, um sicherzustellen, dass der Dienst bereit ist

**Wichtiges SDK-Muster:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Lebenszyklusverwaltung:**
- Manager werden global gespeichert, um eine ordnungsgemäße Bereinigung zu gewährleisten
- Jeder Agent kann ein anderes Modell für Spezialisierungen verwenden
- Automatische Dienstentdeckung und Verbindungsverwaltung
- Sanfte Wiederholungen mit exponentiellem Backoff bei Fehlern

Dies stellt sicher, dass die Initialisierung abgeschlossen ist, bevor die Agentenorchestrierung beginnt.

**Referenz:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Erklärung: Agenten- und Speicherklassen
Definiert den leichtgewichtigen `AgentMsg` für Speicher-Einträge und `Agent`, der Folgendes umfasst:
- **Systemrolle** - Die Persona und Anweisungen des Agenten
- **Nachrichtenverlauf** - Beibehaltung des Gesprächskontexts
- **act()-Methode** - Führt Aktionen mit ordnungsgemäßer Fehlerbehandlung aus

Der Agent kann verschiedene Modelle verwenden (primär vs. Editor) und hält einen isolierten Kontext pro Agent aufrecht. Dieses Muster ermöglicht:
- Speicherpersistenz über Aktionen hinweg
- Flexible Modellzuweisung pro Agent
- Fehlerisolierung und -wiederherstellung
- Einfaches Verketten und Orchestrieren


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Erklärung: Orchestrierte Pipeline
Erstellt zwei spezialisierte Agenten:
- **Forscher**: Nutzt das primäre Modell, sammelt faktische Informationen
- **Redakteur**: Kann ein separates Modell verwenden (falls konfiguriert), verfeinert und überarbeitet

Die Funktion `pipeline`:
1. Der Forscher sammelt Rohinformationen
2. Der Redakteur verfeinert diese zu einem ausgereiften Ergebnis
3. Gibt sowohl die Zwischenergebnisse als auch das Endergebnis zurück

Dieses Muster ermöglicht:
- Spezialisierung der Modelle (verschiedene Modelle für unterschiedliche Rollen)
- Qualitätsverbesserung durch mehrstufige Verarbeitung
- Nachvollziehbarkeit der Informationsumwandlung
- Einfache Erweiterung um weitere Agenten oder parallele Verarbeitung


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Erklärung: Pipeline-Ausführung & Ergebnisse
Führt die Multi-Agenten-Pipeline zu einer Frage mit Schwerpunkt auf Compliance und Latenz aus, um Folgendes zu demonstrieren:
- Mehrstufige Informationsverarbeitung
- Spezialisierung und Zusammenarbeit von Agenten
- Verbesserung der Ausgabequalität durch Verfeinerung
- Nachvollziehbarkeit (sowohl Zwischen- als auch Endergebnisse werden erhalten)

**Ergebnisstruktur:**
- `question` - Ursprüngliche Benutzeranfrage
- `research` - Rohes Forschungsergebnis (faktische Stichpunkte)
- `final` - Verfeinerte Zusammenfassung für Führungskräfte
- `models` - Welche Modelle in jeder Phase verwendet wurden

**Erweiterungsideen:**
1. Hinzufügen eines Kritiker-Agenten zur Qualitätsprüfung
2. Implementierung paralleler Forschungsagenten für verschiedene Aspekte
3. Hinzufügen eines Verifizierungs-Agenten zur Faktenprüfung
4. Verwendung unterschiedlicher Modelle für verschiedene Komplexitätsstufen
5. Implementierung von Feedback-Schleifen für iterative Verbesserungen


### Fortgeschritten: Anpassung der Agentenkonfiguration

Versuchen Sie, das Verhalten des Agenten anzupassen, indem Sie Umgebungsvariablen ändern, bevor Sie die Initialisierungszelle ausführen:

**Verfügbare Modelle:**
- Verwenden Sie `foundry model ls` im Terminal, um alle verfügbaren Modelle anzuzeigen
- Beispiele: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b usw.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
